In [1]:
# Dependencies & Installs
import pandas as pd
import streamlit as st
from joblib import load
from PIL import Image
import numpy as np

In [2]:
local_file_path = "../Data_Cleaned/User_Samples/users.csv"
df = pd.read_csv(local_file_path)

In [3]:
def generate_example_data():
    # Use a sample row from users.csv as the example blood sample data
    sample_row = df.sample(n=1)
    blood_results_df = sample_row.drop(columns=["Patient Group", "Age at Collection", "BMI (kg/m2)", "Gender_F", "Gender_M"])
    return blood_results_df.reset_index(drop=True)

In [4]:
# Load the trained models and scalers
model1 = load('../Models/Model_Saved/Model1.joblib')
model1_X_scaler = load('../Models/Model_Saved/model1_X_scaler.joblib')
model2 = load('../Models/Model_Saved/Model2.joblib')
model2_X_scaler = load('../Models/Model_Saved/model2_X_scaler.joblib')
model3 = load('../Models/Model_Saved/Model3.joblib')
model3_X_scaler = load('../Models/Model_Saved/model3_X_scaler.joblib')
model4 = load('../Models/Model_Saved/Model4.joblib')
model4_X_scaler = load('../Models/Model_Saved/model4_X_scaler.joblib')

In [5]:
# Get user data
age = 55
sex = "male"
height = 180
weight = 90.08
diagnosed = "Barrett esophagus - no/low dysplasia"
unit_system = "Metric"

# Calculate BMI based on unit system
if unit_system == 'Metric':
    bmi = weight / ((height/100)**2) # kg / m^2
elif unit_system == 'Imperial':
    bmi = 703 * (weight / (height**2)) # lb / in^2

# Split sex into binary
gender_f = 1 if sex == "female" else 0
gender_m = 1 if sex == "male" else 0

# Create a DataFrame with the user input
user_input = pd.DataFrame({
    "Age at Collection": [age],
    "BMI (kg/m2)": [bmi],
    "Gender_F": [gender_f],
    "Gender_M": [gender_m]
})

user_input

,Age at Collection,BMI (kg/m2),Gender_F,Gender_M
0,55,27.802469,0,1


In [6]:
# Generate example blood sample data
blood_sample_data = generate_example_data()

In [7]:
model = model4
scaler = model4_X_scaler

In [8]:
# Scale the user input data
user_input_scaled = scaler.transform(user_input)
# If blood sample data is available, append it to the user input
if blood_sample_data is not None:
    user_input_scaled = np.hstack([user_input_scaled, blood_sample_data.to_numpy()])
prediction = model.predict(user_input_scaled)
prediction_proba = model.predict_proba(user_input_scaled)

# Display the prediction
if prediction[0] == 1:
    left_column.write(f"Based on the information you provided and our machine learning model's understanding of the relationship between various factors and oesophageal cancer risk, it is estimated that you have a higher risk of developing oesophageal cancer.\n\n The model predicts a {prediction_proba[0][1]*100:.2f}% probability of you being in the higher-risk group.\n\n Please note that this tool is not a substitute for professional medical advice, diagnosis, or treatment. The results should be considered as an estimate and should not be relied upon for decision-making regarding your health. Always consult with a healthcare professional for personalised medical advice.")
else:
    left_column.write(f"Based on the information you provided and our machine learning model's understanding of the relationship between various factors and oesophageal cancer risk, it is estimated that you have a lower risk of developing oesophageal cancer.\n\n The model predicts a {prediction_proba[0][0]*100:.2f}% probability of you being in the lower-risk group.\n\n Please note that this tool is not a substitute for professional medical advice, diagnosis, or treatment. The results should be considered as an estimate and should not be relied upon for decision-making regarding your health. Always consult with a healthcare professional for personalised medical advice.")

C:\Users\rober\anaconda\Anaconda3\envs\PythonData\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- O75636_NPL
- O95445_AAL
- O95445_JAC
- O95445_NPL
- P00450_AAL
- ...

  warnings.warn(message, FutureWarning)


ValueError: X has 4 features, but StandardScaler is expecting 165 features as input.